In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split # 虽然这里仍然使用，但对于时间序列，推荐手动切分
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, Bidirectional, LSTM, Dropout, Dense
import math
import joblib # 如果需要保存 scaler，请取消注释相关行

In [6]:
forecast_path: str = r'Z:\SOURCE_MERGE_DATA\df_ws_forecast.csv'
realdata_path: str = r'Z:\SOURCE_MERGE_DATA\2024_local_df_utc_183_split.csv'

In [7]:

# --- 0. 加载和初步处理数据 ---
# 读取数据
df_forecast_raw = pd.read_csv(forecast_path, encoding='utf-8', index_col=0)
df_realdata_raw = pd.read_csv(realdata_path, encoding='utf-8', index_col=0)

# 注意：df_realdata_raw 形状是 (61, 731)， df_forecast_raw 是 (61, 732)
# 根据你的注释，这里通常需要对齐列数
# df_realdata_raw = df_realdata_raw.drop(df_realdata_raw.columns[-1], axis=1) # 如果需要对齐
# 这里假设 df_realdata_raw 和 df_forecast_raw 的列数都是 732 或者已经处理好

df_forecast_raw = df_forecast_raw.iloc[:61, :] # 确保行数是61

In [8]:

step = 3
start_hour = 0
end_hour = 96
start_index = math.ceil(start_hour / step) # 0
end_index = math.ceil(end_hour / step)     # 32

# 根据你的逻辑，进行切分，得到 (32, 732) 的数据块
df_forecast_split = df_forecast_raw.iloc[start_index:end_index, :]
df_realdata_split = df_realdata_raw.iloc[start_index:end_index, :]

In [9]:
# **数据预热的核心：从一个连续的真实时间序列中构建 (X, y) 对。**
# 这里我们假设 `df_realdata_split` 的所有值（按列拼接，再按行拼接）构成一个长的连续时间序列。
# 例如：df_realdata_split 的第一列的所有 32 个值，然后是第二列的所有 32 个值，依此类推。
# 如果不是这样，请根据你的数据实际物理含义，构造一个能代表连续时间变化的 `full_time_series`。

# 将 df_realdata_split 的值按列拼接，形成一个很长的单变量序列
# df_realdata_split.values.T 将形状变为 (732, 32)，表示 732 个“时间点”，每个点有 32 个“特征”
# 然后 flatten 就会得到一个长度为 732 * 32 的长序列
full_time_series = df_realdata_split.values.T.flatten() 
total_timesteps = len(full_time_series) # 732 * 32 = 23424

# 定义预热窗口的长度 (这应与你的模型 input_shape=(32,1) 中的 32 匹配)
# 这意味着每个输入序列将包含 32 个历史真实值
look_back_window = 32 

# 存储新的 X 和 y 样本
X_data_for_training, y_data_for_training = [], []

# 遍历 `full_time_series` 来构建滑动窗口样本
# 循环范围：从序列的开始到能够提取最后一个完整的 'look_back_window' 序列并获取其 '下一个' 值
for i in range(total_timesteps - look_back_window):
    # X：过去 'look_back_window' 个时间步的真实值（作为预热数据）
    # 形状为 (look_back_window,)
    seq_in = full_time_series[i : (i + look_back_window)]
    X_data_for_training.append(seq_in)
    
    # y：紧随其后的一个真实值（单步预测目标）
    # 形状为 ()，即单个数值
    target_out = full_time_series[i + look_back_window]
    y_data_for_training.append(target_out)

In [15]:
len(X_data_for_training)

23392

In [10]:

# 将列表转换为 NumPy 数组
X_data_for_training = np.array(X_data_for_training)
y_data_for_training = np.array(y_data_for_training)

# 重塑 X 以适应 LSTM 输入 (样本数, 时间步, 特征数)
X_data_for_training = X_data_for_training.reshape(X_data_for_training.shape[0], look_back_window, 1)
# y_data_for_training 形状已经是 (num_samples,)，适合单步预测，无需额外重塑

print(f"原始 X_data_for_training shape: {X_data_for_training.shape}")
print(f"原始 y_data_for_training shape: {y_data_for_training.shape}")

# --- 1. 数据归一化 ---
# 拍扁 X_data_for_training 以进行归一化
X_flat_for_scaler = X_data_for_training.reshape(-1, 1)
y_flat_for_scaler = y_data_for_training.reshape(-1, 1) # 为了 MinMaxScaler 转换为二维

scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X_scaled_flat = scaler_X.fit_transform(X_flat_for_scaler)
y_scaled_flat = scaler_y.fit_transform(y_flat_for_scaler)

# 将归一化后的数据恢复为原来的形状
X_scaled = X_scaled_flat.reshape(X_data_for_training.shape[0], look_back_window, 1)
y_scaled = y_scaled_flat.flatten() # 恢复为一维数组，因为是单步预测


原始 X_data_for_training shape: (23392, 32, 1)
原始 y_data_for_training shape: (23392,)


In [11]:
# joblib.dump(scaler_X, "path/to/scaler_X.pkl") # 请定义你的路径
# joblib.dump(scaler_y, "path/to/scaler_y.pkl") # 请定义你的路径

In [12]:
# --- 2. 划分训练集和测试集 ---
# 对于时间序列，更推荐手动切分以保持时间顺序
train_size = int(len(X_scaled) * 0.8)
X_train, X_test = X_scaled[0:train_size], X_scaled[train_size:len(X_scaled)]
y_train, y_test = y_scaled[0:train_size], y_scaled[train_size:len(y_scaled)]

print(f"X_train shape: {X_train.shape}") 
print(f"y_train shape: {y_train.shape}") 
print(f"X_test shape: {X_test.shape}")   
print(f"y_test shape: {y_test.shape}")   

# 类型转换和 NaN 处理（保持你的原代码逻辑）
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

X_train = np.nan_to_num(X_train, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)
y_train = np.nan_to_num(y_train, nan=0.0)
y_test = np.nan_to_num(y_test, nan=0.0)

X_train shape: (18713, 32, 1)
y_train shape: (18713,)
X_test shape: (4679, 32, 1)
y_test shape: (4679,)


In [13]:
# --- 3. 模型构建 (核心调整：最后一个LSTM层的 return_sequences) ---
model = Sequential()
# Masking层 input_shape 必须与实际输入序列长度 (look_back_window, 1) 匹配
model.add(Masking(mask_value=0.0, input_shape=(look_back_window, 1))) 

model.add(Bidirectional(LSTM(units=256, return_sequences=True,
                              activation='tanh',
                              input_shape=(look_back_window, 1)))) # 确保 input_shape 与 Masking层一致
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(units=128, return_sequences=True, activation='tanh')))
model.add(Dropout(0.2))

# 关键修改：为了单步预测，最后一个 LSTM 层需要将 return_sequences 设置为 False
# 这样它会输出一个表示整个序列的总结性向量，而不是每个时间步的输出
model.add(Bidirectional(LSTM(units=64, return_sequences=False, activation='tanh'))) 
model.add(Dropout(0.2))

# Dense 层输出单个预测值
model.add(Dense(1)) 

model.compile(optimizer='adam', loss='mse')


In [14]:
# --- 4. 训练模型 ---
model_path: str = r'Z:\02TRAINNING_MODEL\fit_model_v4_spinup_250623.h5'

model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))
model.save(model_path)

Epoch 1/10
1170/1170 [==============================] - 647s 540ms/step - loss: 0.0180 - val_loss: 0.0096
Epoch 2/10
1170/1170 [==============================] - 712s 609ms/step - loss: 0.0157 - val_loss: 0.0100
Epoch 3/10
1170/1170 [==============================] - 632s 540ms/step - loss: 0.0153 - val_loss: 0.0111
Epoch 4/10
1170/1170 [==============================] - 569s 486ms/step - loss: 0.0152 - val_loss: 0.0091
Epoch 5/10
1170/1170 [==============================] - 628s 537ms/step - loss: 0.0149 - val_loss: 0.0095
Epoch 6/10
1170/1170 [==============================] - 636s 544ms/step - loss: 0.0148 - val_loss: 0.0092
Epoch 7/10
1170/1170 [==============================] - 578s 494ms/step - loss: 0.0146 - val_loss: 0.0105
Epoch 8/10
1170/1170 [==============================] - 622s 531ms/step - loss: 0.0146 - val_loss: 0.0114
Epoch 9/10
1170/1170 [==============================] - 653s 558ms/step - loss: 0.0145 - val_loss: 0.0104
Epoch 10/10
1170/1170 [=======================